In [ ]:
import requests
import pandas as pd
from datetime import datetime as dt

In [ ]:
#Script that pulls multiple audio features from our Spotify playlist using API calls

In [ ]:
CLIENT_ID = '1249cb8b97e04bfc8b4de2b1e505993c'
CLIENT_SECRET = '1112a627856942f1b859af480957a140'
PLAYLIST_ID = '2YRe7HRKNRvXdJBp9nXFza'
import requests


# Declare the global variable
track_info_list = []

def get_access_token():
    response = requests.post('https://accounts.spotify.com/api/token', 
                             data={'grant_type': 'client_credentials'}, 
                             auth=(CLIENT_ID, CLIENT_SECRET))
    return response.json()['access_token']

def get_audio_features(track_ids, token):
    audio_features = []
    for i in range(0, len(track_ids), 100):  # Process in batches of 100
        batch_ids = track_ids[i:i + 100]
        url = f'https://api.spotify.com/v1/audio-features?ids={",".join(batch_ids)}'
        response = requests.get(url, headers={'Authorization': f'Bearer {token}'})
        if response.status_code == 200:
            audio_features.extend(response.json()['audio_features'])
        else:
            print("Error fetching audio features:", response.status_code)
            return []
    return audio_features

def get_artist_info(artist_id, token):
    url = f'https://api.spotify.com/v1/artists/{artist_id}'
    response = requests.get(url, headers={'Authorization': f'Bearer {token}'})
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching artist info:", response.status_code)
        return {}

def get_album_info(album_id, token):
    url = f'https://api.spotify.com/v1/albums/{album_id}'
    response = requests.get(url, headers={'Authorization': f'Bearer {token}'})
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching album info:", response.status_code)
        return {}

def get_playlist_tracks(playlist_id, token):
    global track_info_list  # Declare the use of the global variable
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    track_ids = []

    while url:
        response = requests.get(url, headers={'Authorization': f'Bearer {token}'})
        if response.status_code == 200:
            data = response.json()
            for item in data['items']:
                track = item['track']
                if track:
                    album = track['album']
                    artist_info = get_artist_info(track['artists'][0]['id'], token)
                    genre = get_album_info(album['id'], token).get('genres', [])
                    
                    track_info = {
                        'name': track['name'],
                        'artist': ', '.join(artist['name'] for artist in track['artists']),
                        'popularity': track['popularity'],
                        'id': track['id'],  # Store track ID for fetching audio features
                        'release_date': album['release_date'],
                        'track_number': track['track_number'],
                        'explicit': track['explicit'],
                        'duration_ms': track['duration_ms'],  # Duration in milliseconds
                        'artist_popularity': artist_info.get('popularity', 0),
                    }
                    track_info_list.append(track_info)  # Append to the global variable
                    track_ids.append(track['id'])  # Collect track IDs for audio features
            url = data['next']  # Get the URL for the next page
        else:
            print("Error fetching playlist tracks:", response.status_code)
            break

    # Get audio features after fetching all tracks
    audio_features = get_audio_features(track_ids, token)
    for i, features in enumerate(audio_features):
        if features:  # Only add features if they were successfully retrieved
            track_info_list[i].update({
                'danceability': features['danceability'],
                'energy': features['energy'],
                'key': features['key'],
                'loudness': features['loudness'],
                'mode': features['mode'],
                'speechiness': features['speechiness'],
                'acousticness': features['acousticness'],
                'instrumentalness': features['instrumentalness'],
                'liveness': features['liveness'],
                'valence': features['valence'],
                'tempo': features['tempo'],
            })

# Main execution
if __name__ == "__main__":
    token = get_access_token()
    get_playlist_tracks(PLAYLIST_ID, token)
    for track in track_info_list:
        print(track)  # Print all track information, including audio features


In [ ]:
df = pd.DataFrame(track_info_list)

In [ ]:
df.to_csv('spotify_df', index=False)